In [71]:
import requests

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0"
}

In [72]:
def get_ticker_news(ticker):
    url = f"https://www.google.com/finance/quote/{ticker}"
    r = requests.get(url, headers=headers)
    return r.text

In [73]:
from bs4 import BeautifulSoup
import nltk
import ssl

# ssl
try:
  _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
  pass
else:
  ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/joeltan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [74]:
import re
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def prepare_text(text: str):
    text = text.lower().strip()
    # remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    text = " ".join([i for i in text.split() if i not in stop_words])

    return text

In [75]:
def get_headlines(soup):
    headlines_text = []
    headlines = soup.find_all("div", {"class": "F2KAFc"})
    for headline in headlines:
        headlines_text.append(prepare_text(headline.text))
    return headlines_text

In [97]:
def get_change(soup: BeautifulSoup):
    current = soup.find("div", {"class": "YMlKec fxKbKc"})
    previous_close = soup.find("div", {"class": "P6K39c"})
    if not current or not previous_close:
        return None
    current = current.text
    previous_close = previous_close.text

    # remove currency symbol
    current = current.replace("$", "").replace(",", "")
    previous_close = previous_close.replace("$", "").replace(",", "")
    
    # turn to numbers
    current_f = float(current)
    previous_close_f = float(previous_close)
    change = current_f - previous_close_f
    change_percent = (change / previous_close_f) * 100
    
    return change_percent

In [89]:
tickers = open("ticker-symbols.txt", "r").read().splitlines()

tickers

['AAPL:NASDAQ',
 'MSFT:NASDAQ',
 'AMZN:NASDAQ',
 'GOOGL:NASDAQ',
 'FB:NASDAQ',
 'TSLA:NASDAQ',
 'NVDA:NASDAQ',
 'INTC:NASDAQ',
 'CSCO:NASDAQ',
 'ADBE:NASDAQ',
 'IBM:NYSE',
 'ORCL:NYSE',
 'QCOM:NASDAQ',
 'CRM:NYSE',
 'PYPL:NASDAQ',
 'SQ:NYSE',
 'AMD:NASDAQ',
 'NFLX:NASDAQ',
 'SHOP:NYSE',
 'ZM:NASDAQ',
 'TWTR:NYSE',
 'INTC:NASDAQ',
 'MU:NASDAQ',
 'WDC:NASDAQ',
 'AMAT:NASDAQ',
 'HPQ:NYSE',
 'DELL:NYSE',
 'AVGO:NASDAQ',
 'TXN:NASDAQ',
 'ASML:NASDAQ',
 'EA:NASDAQ',
 'ATVI:NASDAQ',
 'AMD:NASDAQ',
 'WDAY:NASDAQ',
 'NOW:NYSE',
 'INTU:NASDAQ',
 'VMW:NYSE',
 'STX:NASDAQ',
 'TWLO:NYSE',
 'DOCU:NASDAQ',
 'CSCO:NASDAQ',
 'ADI:NASDAQ',
 'NOK:NYSE',
 'JNPR:NYSE',
 'FFIV:NASDAQ',
 'PANW:NYSE',
 'FTNT:NASDAQ',
 'ANET:NYSE',
 'AKAM:NASDAQ',
 'ZS:NASDAQ']

In [90]:
ticker_soups = []

for ticker in tickers:
  ticker_news = get_ticker_news(ticker)
  soup = BeautifulSoup(ticker_news, "html.parser")
  ticker_soups.append(soup)

In [98]:
for i, soup in enumerate(ticker_soups):
  headlines: list[str] = get_headlines(soup)
  change = get_change(soup)
  headline_str = " ".join(headlines)

  if headline_str:
    print(f"{tickers[i]}, {headline_str}, {change}")

AAPL:NASDAQ, best worst positioned mag 7 stocks coming earnings india apples wearable laptop tablet business fails match iphones impressive growth 2023 sp 500 earnings still solid nvidia walmart look apple heres shareholders may want cautious increasing apple incs nasdaqaapl ceo pay packet vision pro review apples first headset lacks polish purpose working parents review apple vision pro apple make hard share vision pro apple vision pro review revolution progress apple byjus ceos fired companies memo employees tim cook used 2 words demonstrate important principle every leader increases ceo compensation might put hold apple inc nasdaqaapl, -1.0034170418180801
MSFT:NASDAQ, microsoft stock 2024 outlook 3 strong catalysts growth nasdaqmsft 2 revolutionary stocks id buy right without hesitation share microsoft 365 subscription microsoftbacked openai valued 80bn company completes deal indianorigin ceo quit microsoft launch startup got rs 25 crore deal shark tank us 29yearold ceo quit microso